In [6]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=bfa1b6f4a05aa751ac3263eb47521f551ceb72eaf5400ec3676776b71db86a3a
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [7]:
#Imports
import os
import sys
import glob
import torch
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser


%matplotlib inline

In [8]:
#/kaggle/input/ammi-2023-convnets


data_path = "/kaggle/input/ammi-2023-convnets"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")

In [9]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
# train_transforms = transforms.Compose([transforms.RandomRotation(30),
#                                        transforms.RandomResizedCrop(224),
#                                        transforms.RandomHorizontalFlip(),
#                                        transforms.ToTensor(),
#                                        transforms.RandomErasing(0.1)])

# test_transforms = transforms.Compose([ transforms.Resize(255),
#                                        transforms.CenterCrop(224),
#                                        transforms.ToTensor()])

# normalize = transforms.Normalize(mean=mean, std=std)


train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(550),
                                       transforms.RandomCrop(500),
                                       transforms.RandomHorizontalFlip(0.3),
                                       transforms.RandomVerticalFlip(0.3),
                                       transforms.ToTensor(),
                                       transforms.RandomErasing(0.1),
                                       transforms.Normalize(mean=mean, std=std)])

test_transforms = transforms.Compose([ transforms.Resize(550),
                                      transforms.CenterCrop(500),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])


# normalize = transforms.Normalize(mean=mean, std=std)

In [10]:
# class CassavaDataset(Dataset):
#     def __init__(self, path, transform=None):
#         self.classes = os.listdir(path)
#         self.path = [f"{path}/{className}" for className in self.classes]
#         self.file_list = [glob.glob(f"{x}/*") for x in self.path]
#         self.transform = transform

#         files = []
#         for i, className in enumerate(self.classes):
#             for fileName in self.file_list[i]:
#                 files.append([i, className, fileName])
#         self.file_list = files
#         files = None

#     def __len__(self):
#         return len(self.file_list)

#     def __getitem__(self, idx):
#         fileName = self.file_list[idx][2]
#         classCategory = self.file_list[idx][0]
#         im = Image.open(fileName)
#         if self.transform:
#             im = self.transform(im)
            
#         return im.view(3, 224, 224), classCategory

In [11]:

class CassavaDatasetNew(Dataset):
    
    def __init__(self, path, transform=None, augment_transform= None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.augment_transform=augment_transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return 2* len(self.file_list) if self.augment_transform else len(self.file_list)

    def __getitem__(self, idx):
        im = None
        if self.augment_transform:
            fileName = self.file_list[idx//2][2]
            classCategory = self.file_list[idx//2][0]
            im = Image.open(fileName)
            if idx%2 == 1:
                im = self.augment_transform(im)
            elif self.transform:
                im = self.transform(im)
        else:
            fileName = self.file_list[idx][2]
            classCategory = self.file_list[idx][0]
            im = Image.open(fileName)
            if self.transform:
                im = self.transform(im)
        return im.view(3, 500, 500), classCategory

In [12]:
train_data = CassavaDatasetNew(train_path, transform=train_transforms, augment_transform=test_transforms)
test_data = CassavaDatasetNew(test_path, transform=test_transforms)

In [13]:
validation_split = .1
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

validation_split = 0.2
random_seed = 42

# Creating data indices for training and validation splits using stratified splitting:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=validation_split, random_state=random_seed)
labels = [train_data[i][1] for i in range(len(train_data))]
# Perform the stratified splitting
train_indices, val_indices = next(splitter.split(train_data,labels))

# Print the sizes of the training and validation sets
print(f"Training set size: {len(train_indices)}")
print(f"Validation set size: {len(val_indices)}")

In [16]:
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8)

In [17]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [18]:
# # Define Models 

# class Classifier(nn.Module):
#     def __init__(self, num_classes):
#         super(Classifier, self).__init__()
#         # Block 1
#         self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 5, stride = 2, padding = 1)
#         self.relu1 = nn.ReLU()
#         self.maxpool1 = nn.MaxPool2d(kernel_size = 2)

#         #Block 2
#         self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, stride = 2)
#         self.relu2 = nn.ReLU()
#         self.maxpool2 = nn.MaxPool2d(kernel_size=2)

#         #Block 3
#         self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 2)
#         self.relu3 = nn.ReLU()
#         self.maxpool3 = nn.MaxPool2d(kernel_size=2)

#         # last fully-connected layer
#         self.fc = nn.Linear(32*3*3, num_classes)


#     def forward(self, input):

#         x = self.maxpool1(self.relu1(self.conv1(input)))
#         x = self.maxpool2(self.relu2(self.conv2(x)))
#         x = self.maxpool3(self.relu3(self.conv3(x)))

#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x


# model = Classifier(5)

In [19]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.num_classes = num_classes
        
        # Load the pretrained EfficientNet model
        self.model = EfficientNet.from_pretrained('efficientnet-b0')
        
        # Replace the classifier head with a new one
        num_ftrs = self.model._fc.in_features
        self.model._fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, input):
        return self.model(input)

In [20]:
model = Classifier(5)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 180MB/s]

Loaded pretrained weights for efficientnet-b0


In [21]:
!pip install git+https://github.com/gbaydin/hypergradient-descent.git

  Cloning https://github.com/gbaydin/hypergradient-descent.git to /tmp/pip-req-build-sedd5bu6
  Running command git clone --filter=blob:none --quiet https://github.com/gbaydin/hypergradient-descent.git /tmp/pip-req-build-sedd5bu6
  Resolved https://github.com/gbaydin/hypergradient-descent.git to commit 020d6080c4cedfbc88d5cdb7a2a53f92b34c2b16
  Preparing metadata (setup.py) ... done
  Created wheel for hypergrad: filename=hypergrad-0.1-py3-none-any.whl size=8199 sha256=d0bdd5c649150a902d0fdc2cf7c75de78604f35308fa3a9c9724473f25c54e97
  Stored in directory: /tmp/pip-ephem-wheel-cache-2g5osiqe/wheels/12/18/2f/fc715468b5745149d6d37c67b27ef8a4f5de487c90d71576b5
Successfully built hypergrad


In [22]:
from hypergrad import SGDHD, AdamHD

In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamHD(model.parameters(), lr=0.001,hypergrad_lr=1e-9,weight_decay =1e-4)

In [24]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in range(num_epochs):
        
      # Loop over data.
      for batch_idx, (features, target) in enumerate(data_loader):
            
          # Forward pass.
        output = model(features.to(device))
        loss = criterion(output.to(device), target.to(device))

          # Backward pass.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      # NOTE: It is important to call .item() on the loss before summing.
        if ema_loss is None:
            ema_loss = loss.item()
        else:
            ema_loss += (loss.item() - ema_loss) * 0.01 

      # Print out progress the end of epoch.
      print('Epoch: {} \tLoss: {:.6f}'.format(epoch, ema_loss),)
  

In [25]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0
    print('----- Model Evaluation -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, target in data_loader:
          
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100. * correct / len(valid_sampler)
    print(f'Test accuracy: {correct} / {len(valid_sampler)} ({percent:.0f}%)')
    torch.save(model.state_dict(), 'model.ckpt')
    return percent

In [26]:
# torch.cuda.empty_cache()

In [27]:
num_epochs = 10
train(model, criterion, train_loader, optimizer, num_epochs=num_epochs)
test(model, valid_loader)

----- Training Loop -----


/opt/conda/lib/python3.10/site-packages/hypergrad/adam_hd.py:71: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1485.)
  grad = grad.add(group['weight_decay'], p.data)


Epoch: 0 	Loss: 0.546102
Epoch: 1 	Loss: 0.479684
Epoch: 2 	Loss: 0.398433
Epoch: 3 	Loss: 0.411472
Epoch: 4 	Loss: 0.408007
Epoch: 5 	Loss: 0.363686
Epoch: 6 	Loss: 0.374313
Epoch: 7 	Loss: 0.326703
Epoch: 8 	Loss: 0.335908
Epoch: 9 	Loss: 0.303429
----- Model Evaluation -----
Test accuracy: 974 / 1131 (86%)


86.1184792219275

In [28]:
import tqdm

def predict(model, loader):
    model.eval()
    model.to(device)
    
    test_dataloader =  loader
    preds = []
    
    for test_images,test_labels in (test_dataloader):
        #print(test_images)
        test_images = test_images.to(device)
        #test_labels = test_labels.to(device)
      
        output = model(test_images)
    

        _, predicted = torch.max(output, 1)
        preds.extend(predicted.cpu().data.numpy())
        
        
    return preds

In [29]:
name = [test_data.file_list[i][-1].split('/')[-1] for i in range(len(test_data.file_list)) ]

In [30]:
import pandas as pd

sample = pd.read_csv('/kaggle/input/ammi-2023-convnets/sample_submission_file.csv')
sample=pd.DataFrame()

preds = predict(model, test_loader)


In [31]:
predict1 = np.array(preds).reshape((1,-1)).flatten()

import pandas as pd
pd.Series(predict1).value_counts()

0    1586
2    1000
4     758
3     231
1     199
dtype: int64

In [32]:
train_data.classes

['cmd', 'cbb', 'cbsd', 'healthy', 'cgm']

In [34]:
mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}
new_preds = [mapping[pred] for pred in preds]
sample['Category'] = preds
sample['Id'] = name
sample.to_csv('jj.csv', index=False)
sample.head()

,Category,Id
0,cgm,cgm
1,cmd,cmd
2,cmd,cmd
3,cmd,cmd
4,cgm,cgm


In [36]:
import pandas as pd

name = [test_data.file_list[i][-1].split('/')[-1] for i in range(len(test_data.file_list)) ]
sample = pd.read_csv('/kaggle/input/ammi-2023-convnets/sample_submission_file.csv')

sample['Id'] = name

mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}

new_preds = [mapping[int(pred)] for pred in predict1]


sample['Category'] = new_preds 


sample.to_csv('submission_ijjjj.csv', index=False)


sample.head()

,Category,Id
0,cgm,test-img-1448.jpg
1,cmd,test-img-768.jpg
2,cmd,test-img-3481.jpg
3,cmd,test-img-1475.jpg
4,cgm,test-img-2498.jpg


In [33]:
# import the modules we'll need
from IPython.display import HTML
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "submit.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# create a link to download the dataframe
create_download_link(sample)

In [28]:
train_data.classes

['cbsd', 'cbb', 'cgm', 'cmd', 'healthy']

In [ ]:
# Make submission here

In [ ]:
pd.Series(preds).value_counts()

2    1781
1     893
4     576
0     292
3     232
dtype: int64

In [ ]:
mapping= {0: 'cbb', 1: 'cbsd', 2: 'cmd', 3: 'healthy', 4: 'cgm'}
new_preds = [mapping[pred] for pred in preds]
sample['Category'] = new_preds
sample['Id'] = name
sample.to_csv('lio_10_05_23_new.csv', index=False)
sample.head(20)

,Category,Id
0,cmd,test-img-3552.jpg
1,healthy,test-img-359.jpg
2,cmd,test-img-3470.jpg
3,cmd,test-img-3572.jpg
4,healthy,test-img-3503.jpg
5,cbsd,test-img-3593.jpg
6,cbb,test-img-353.jpg
7,cbsd,test-img-3508.jpg
8,cmd,test-img-3472.jpg
9,cmd,test-img-3555.jpg
